In [1]:
import torch
import torch.autograd
from torch.autograd import Variable

def mlp(x, Weight, Bias, Func):
    f = Variable(x, requires_grad=False)
    #print(f,"\n\n")
    #print(Weight[0],"\n\n")
    #print(Func[0](torch.matmul(Weight[0], f) + Bias[0]),"\n\n")
    NumOfLayers = len(Weight)
    for i in range(NumOfLayers):
        f = Func[i](torch.matmul(Weight[i], f) + Bias[i])
        #print(f,"\n\n")
    return f

## Part 1

In [2]:
sizes = [1,2,1]

x = 10*torch.rand(sizes[0]).double()

Weight = []
Bias = []
Func = []

for i in range(len(sizes)-1):
    # For layer i, Weights are a S_{i+1} \times S_{i} matrix
    W = Variable(torch.randn(sizes[i+1], sizes[i]).double(), requires_grad=True)
    # For layer i, Biases are a S_{i+1} \times 1 matrix (a vector)
    b = Variable(torch.randn(sizes[i+1], 1).double(), requires_grad=True)
    
    Weight.append(W)
    Bias.append(b)
    Func.append(torch.sigmoid)

f = mlp(x, Weight, Bias, Func)

print(f)

tensor([[0.4009, 0.2272]], dtype=torch.float64, grad_fn=<SigmoidBackward>)


## Part 2

In [3]:
sizes = [4, 2, 1]

x = 10*torch.rand(sizes[0]).double()

Weight = []
Bias = []
Func = []

for i in range(len(sizes)-1):
    # For layer i, Weights are a S_{i+1} \times S_{i} matrix
    W = Variable(torch.randn(sizes[i+1], sizes[i]).double(), requires_grad=True)
    # For layer i, Biases are a S_{i+1} \times 1 matrix (a vector)
    b = Variable(torch.randn(sizes[i+1], 1).double(), requires_grad=True)
    
    Weight.append(W)
    Bias.append(b)
    Func.append(torch.sigmoid)
    
f = mlp(x, Weight, Bias, Func)

print(f)

tensor([[0.2615, 0.3297]], dtype=torch.float64, grad_fn=<SigmoidBackward>)


### İris Dataset

In [6]:
import pandas as pd
import numpy as np

def logsumexp(b):
    return torch.log(1 + torch.exp(b))

def mlp(x, Weight, Bias, Func):
    f = Variable(x, requires_grad=False) # Initializing f with x

    # f = sigmoid(w.dot(x) + b)
    for i in range(len(Weight)):
        f = torch.sigmoid(torch.matmul(Weight[i], f) + Bias[i])
        
    return f

# 3 layers: 4 inputs-2 hidden variables-1 output
sizes = [4, 2, 1]

df_iris = pd.read_csv(u'data/iris.txt',sep=' ')

# Get İris data targets, set values bigger than 1 to 0
y = torch.from_numpy(np.array(df_iris['c'])).double()
y[y>1]=0

# Get İris data features
x = torch.from_numpy(np.array(df_iris[['sl','sw','pl','pw']])).double()

Weight = []
Bias = []

# Initializing Weight and Bias with random values
for i in range(len(sizes)-1):
    # For layer i, Weights are a sizes[i+1] x sizes[i] matrix
    W = Variable(20*torch.randn(sizes[i+1], sizes[i]).double(), requires_grad=True)
    
    # For layer i, Biases are a sizes[i+1] x 1 vector
    b = Variable(torch.randn(sizes[i+1], 1).double(), requires_grad=True)
    
    Weight.append(W)
    Bias.append(b)

# Euclidian cost error function
Error = torch.nn.MSELoss(size_average=True)
    
# Iteration variables
eta = 0.005
MAX_ITER = 1000
EE = []

for epoch in range(MAX_ITER):  
    for i in range(len(y)):
        f = mlp(x[i,:], Weight, Bias, Func)

        # Measure the error
        E = Error(f[-1][-1], y[i])
        EE.append(E.data.numpy())

        # Compute the derivative of the error with respect to Weights and Biases
        E.backward() 

        # Take the step and reset weights
        for j in range(len(sizes)-1):
            Weight[j].data.add_(-eta*Weight[j].grad.data)
            Bias[j].data.add_(-eta*Bias[j].grad.data)
            Weight[j].grad.zero_()
            Bias[j].grad.zero_()
    
    if(epoch%100==0):
        print("In epoch ", epoch, ", EE is:", EE[epoch])

result = torch.zeros(len(y))
        
for i in range(len(y)):
    result[i] = torch.round(mlp(x[i,:], Weight, Bias, Func)[-1][-1])

confusion_matrix = torch.zeros((2,2))

for i in range(len(result)):
    confusion_matrix[int(y[i] - 1), int(result[i] - 1)] += 1
    
print("Confusion matrix:\n", confusion_matrix, "\n")
    
accuracy = torch.sum(torch.diag(confusion_matrix))/torch.sum(confusion_matrix)

print("Accuracy:", accuracy*100, "%\n")

print("Final Error is:", EE[-1])

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))


In epoch  0 , EE is: 0.9675139198736301
In epoch  100 , EE is: 0.0002809652970227631
In epoch  200 , EE is: 0.00029463459122299265
In epoch  300 , EE is: 0.965993560449552
In epoch  400 , EE is: 0.00030875439241159254
In epoch  500 , EE is: 0.00032447310903033796
In epoch  600 , EE is: 0.964331360854156
In epoch  700 , EE is: 0.0003407272044038618
In epoch  800 , EE is: 0.0003589139742345957
In epoch  900 , EE is: 0.9625073283455486
Confusion matrix:
 tensor([[  0.,  50.],
        [  0., 100.]]) 

Accuracy: tensor(66.6667) %

Final Error is: 0.09595984891197484
